In [1]:
!pwd

/notebooks/notebooks/SnowparkRnD/UDFs


In [3]:
from Run_Experiment import run_experiment

In [8]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf

In [9]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [10]:
session.add_import("tests/resources/test_udf_dir/test_udf_file.py", import_path="resources.test_udf_dir.test_udf_file")
session.add_packages("numpy")

FileNotFoundError: tests/resources/test_udf_dir/test_udf_file.py is not found